<a href="https://colab.research.google.com/github/snghynn/summ/blob/main/%ED%8C%8C%EB%A8%B8%EC%99%84_ch8_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pg. 487~496

## 8.4 텍스트 분류 실습 - 20 뉴스그룹 분류
- 텍스트를 피처 벡터화로 변화하면 일반적으로 희소 행렬 형태가 됨
- 텍스트 정규화한 뒤 피처 벡터화 적용
- 이후 적합한 머신러닝 알고리즘 적용해 분류를 학습/예측/평가

### 텍스트 정규화

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

- fetch_20newsgroups(): 딕셔너리와 유사한 Bunch 객체 반환

In [ ]:
# key 값들 확인
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


- filenames: fetch_20newsgroups() API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명 지칭


In [ ]:
# target 클래스 구성 확인
import pandas as pd
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


>- target 클래스의 값: 0~19(20개로 구성)

In [ ]:
# 개별 데이터가 텍스트로 어떻게 구성돼 있는지 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

>- 뉴스그룹 기사의 내용, 제목, 작성자, 소속, 이메일 등의 다양한 정보 있음
- remove 파라미터: 내용 제외한 다른 정보 제거(제목, 소속 등의 정보들은 뉴스그룹 분류의 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문)
- 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류


In [ ]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용(Train) 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news= fetch_20newsgroups(subset = 'train',
                               remove = ('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))


test_news= fetch_20newsgroups(subset = 'test',
                              remove = ('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 학습 데이터: 11314개의 뉴스그룹 문서가 리스트 형태로 주어짐
- 테스트 데이터: 7532개의 문서가 리스트 형태로 주어짐
- CountVectorizer: 학습 데이터의 텍스트를 피처 벡터화
  - fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환(transform) 해야함
- 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform() 이용해야 함
  - fit_transform 사용하면 안됨

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)
print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- test data 역시 피처 벡터화 수행
- test data에서 CountVectorier를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 transform해야함
- 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐

- test data의 피처 벡터화 시 fit_transform() 사용하면 안됨

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터 feature extraction 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


>- 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11314개의 문서에 피처(단어)가 101631개로 만들어짐

피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류 예측

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression(solver = 'liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.617


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


> - Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도는 0.617

- Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization을 적용하여 학습 데이터셋과 테스트 데이터세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression(solver = 'liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.678


> - TF-IDF가 단순 카운트 기반보다 높은 예측 정확도
- 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 TF-IDF 벡터화가 좋은 예측 결과 도출

- 텍스트 분석에서 ML model의 성능 향상시키는 중요한 두가지 방법
  1. 최적의 ML 알고리즘을 선택
  2. 최상의 피처 전처리 수행

- 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 적용했는지가 텍스트 기반의 ML 성능에 큰 영향 미침

<다양한 파라미터 적용>
- stop_words = 'english'
- ngram_range = (1,2)
- max_df = 300

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver = 'liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


<GridSearchCV 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화 수행>
- C 파라미터만 변경하면서 최적의 C값을 찾은 뒤 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3 폴드 세트로 설정
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- Pipeline: 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 '수도관에서 물이 흐르듯' 한꺼번에 스트림 기반으로 처리
- 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리 -> 더 직관적인 ML 모델 코드를 생성
- 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 ML 알고리즘의 데이터로 입력 가능: 수행 시간 절약


In [ ]:
from sklearn.pipeline import Pipeline

# 위의 텍스트 분류 예제 코드를 Pipeline을 이용해 작성
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
                     'lr_clf', LogisticRegression(random_state = 156)])
# TfidVectorizer 객체를 tfidf_vect라는 객체 변수명으로, LogisticRegression 객체를 lr_clf 객체 변수명으로 생성
# 이 두개의 객체를 pipeline으로 연결하는 Pipeline 객체 pipeline을 생성

- TfidfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit()과 transform() 수행을 통한 피처 벡터화와 LogisticRegressor의 fit()과 predict() 수행을 통한 머신러닝 모델의 학습과 예측이 Pipeline의 fit()과 predict()로 통일돼 수행됨

In [ ]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english', ngram_range=(1,2), max_df = 300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer 객체의 fit(), tranform()과 LogisticRegression의 fit(), predict()개 필요없음
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도은 0.701


- Pipeline 기반에서도 하이퍼 파라미터 튜닝을 GridSearchCV 방식으로 진행 가능
- 피처 벡터화 위한 파라미터와 하이퍼 파라미터를 한번에 GridSearchCV를 이용해 최적화 가능

#### 예제)
- GridSearchCV에 Pipeline을 입력하면서 TfidfVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화

- 개별 객체명과 파라미터명/하이퍼파라미터명을 결합해 Key 값으로 할당
  - tfidf_vect + '__' + ngram_range
  - tfidf_vect__ngram_range
- 주의점
  - 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모됨
  - 피처 벡터화에 사용되는 파라미터와 GridSearchCV 하이퍼파라미터를 합치면 최적화를 위한 너무 많은 경우의 수가 발생


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression(solver = 'liblinear'))
])

params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.7550828826229531
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.702
